In [3]:
import ee

import geemap
# from sar_change_detection.settings import env, BASE_DIR
import os
import json
# from datetime import datetime
# import numpy as np
# import torch
# import torchvision.transforms as transforms
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.cluster import DBSCAN
from segmentation_models_pytorch import DeepLabV3Plus
from PIL import Image
# import rasterio
# from rasterio.features import shapes
# from shapely.geometry import shape
# import geopandas as gpd

import json

# Initialize Earth Engine
service_account = "django@sih-sar-prathameshks2003.iam.gserviceaccount.com"
credentials = ee.ServiceAccountCredentials(service_account, "D:\Coding\sih-sar-project\sar_change_detection\service-account-key.json")
ee.Initialize(credentials)



<>:24: SyntaxWarning: invalid escape sequence '\C'
<>:24: SyntaxWarning: invalid escape sequence '\C'


C:\Users\Dell\AppData\Local\Temp\ipykernel_7632\1261469072.py:24: SyntaxWarning: invalid escape sequence '\C'
  credentials = ee.ServiceAccountCredentials(service_account, "D:\Coding\sih-sar-project\sar_change_detection\service-account-key.json")


In [4]:

def detect_changes():
    if True:
        # try :
        if True:
            # Parse input parameters
           
            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]} 
            start_date =  "2024-08-01"
            end_date = "2024-09-28"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            # Load Sentinel-2 imagery
            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get start and end images
            s2_start = s2_collection.filterDate(start_date, ee.Date(start_date).advance(1, 'month')).median()
            s2_end = s2_collection.filterDate(ee.Date(end_date).advance(-1, 'month'), end_date).median()

            # Perform change detection
            def detect_change(current, previous):
                previous_img = ee.Image(previous)
                diff = current.select(['B4', 'B3', 'B2']).subtract(previous_img.select(['B4', 'B3', 'B2'])).abs()
                return ee.Image(previous).addBands(diff.reduce(ee.Reducer.sum()).rename('change'))


            first = s2_collection.sort('system:time_start').first()
            
            # changes_list = ee.List(s2_collection.sort('system:time_start').iterate(detect_change, first))
           
            changes_obj = s2_collection.sort('system:time_start').iterate(detect_change, first)        
            changes = ee.ImageCollection.fromImages(changes_obj)
            
            # changes = ee.ImageCollection(s2_collection.sort('system:time_start').iterate(detect_change, first))
            
            change_image = changes.select('change').max()

            # Apply threshold to identify significant changes
            threshold = 0.1  # Adjust this value based on your needs
            significant_changes = change_image.gt(threshold)

            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson
            }

            return response

print(detect_changes())

EEException: ImageCollection.fromImages: Parameter 'images' is required.